In [1]:
from transformers import HfArgumentParser

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from modeling_icae_multi_span import ICAE, ModelArguments, DataArguments, TrainingArguments

2025-02-15 09:34:12.411649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739612052.441668    2967 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739612052.450721    2967 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import transformers
parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))

In [4]:
parser

HfArgumentParser(prog='ipykernel_launcher.py', usage=None, description=None, formatter_class=<class 'argparse.ArgumentDefaultsHelpFormatter'>, conflict_handler='error', add_help=True)

In [5]:
model_args, data_args, training_args = parser.parse_args_into_dataclasses()

usage: ipykernel_launcher.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                             [--lora_r LORA_R] [--lora_dropout LORA_DROPOUT]
                             [--train [TRAIN]] [--no_train]
                             [--data_path DATA_PATH]
                             [--debug_data [DEBUG_DATA]] --output_dir
                             OUTPUT_DIR
                             [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                             [--do_train [DO_TRAIN]] [--do_eval [DO_EVAL]]
                             [--do_predict [DO_PREDICT]]
                             [--eval_strategy {no,steps,epoch}]
                             [--prediction_loss_only [PREDICTION_LOSS_ONLY]]
                             [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
                             [--per_device_eval_batch_size PER_DEVICE_EVAL_BATCH_SIZE]
                             [--per_gpu_train_batch_size PER_GPU_TRAIN_BATCH_SIZE]
               

SystemExit: 2

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# OK

In [1]:
import transformers

In [2]:
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class ModelArguments:
    model_name_or_path: str = field(default="mistralai/Mistral-7B-v0.1")
    lora_r: int = field(
        default=128,
        metadata={"help": "lora rank"}
    )
    lora_dropout: float = field(
        default=0.05,
        metadata={"help": "lora dropout"}
    )
    train: bool = field(
        default=True,
        metadata={"help": "if true, the model ckpt will be initialized for training; else, it's for inference"}
    )

@dataclass
class DataArguments:
    data_path: str = field(default=None, metadata={"help": "Path to the training data."})
    debug_data: bool = field(default=False, metadata={"help": "Enable debug dataset to quickly verify the training process"})

@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=28000,
        metadata={"help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."},
    )
    fixed_mem_size: int = field(
        default=128,
        metadata={"help": "Enalbing the fixed mem size."},
    )
    mean_compression_rate: int = field(
        default=4,
        metadata={"help": "Mean compression rate; default=4"},
    )
    min_tokens_for_lm: int = field(
        default=64,
        metadata={"help": "Minimum tokens for lm objective learning"},
    )
    leave_tokens_for_lm: int = field(
        default=8,
        metadata={"help": "Leave some tokens without loss for lm objective"},
    )
    lm_ratio: float = field(
        default=0.0,
        metadata={"help": "Ratio for LM training."},
    )
    add_special_token_for_lm: bool = field(
        default=False,
        metadata={"help": "Add a special token for the prompt of language modeling; default: False"},
    )
    restore_from: str = field(
        default="",
        metadata={"help": "The checkpoint that should be restored from for fine-tuning"}
    )

In [3]:
ModelArguments(), TrainingArguments(output_dir="./output"), DataArguments()

2025-02-18 05:22:59.232518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739856179.254742    2489 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739856179.261374    2489 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(ModelArguments(model_name_or_path='mistralai/Mistral-7B-v0.1', lora_r=128, lora_dropout=0.05, train=True),
 TrainingArguments(output_dir='./output', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='./output/runs/Feb18_05-22-57_104-171-202-244', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, loggin

In [52]:
from datasets import load_dataset

ds = load_dataset("ankner/gsm8k-CoT")
train_dataset = ds["train"]
eval_dataset = ds["test"]

In [53]:
train_dataset = train_dataset.map(lambda example: {**example, "text": f"{example['question']}\n{example['response']}"})
eval_dataset = eval_dataset.map(lambda example: {**example, "text": f"{example['question']}\n{example['response']}"})

In [54]:
def extract_reasoning_trace(response):
    """
    Extracts the reasoning trace from a response by splitting exactly at "Therefore,".

    Args:
        response (str): The full response including reasoning and the final answer.

    Returns:
        str: The extracted reasoning trace without the final answer.
    """
    # Define the exact split phrase
    split_phrase = "Therefore,"

    # Search for the exact occurrence of "Therefore,"
    match = response.find(split_phrase)

    if match != -1:
        reasoning_trace = response[:match].strip()  # Keep everything before "Therefore,"
    else:
        reasoning_trace = ""  # If not found, return the full response

    return reasoning_trace


def preprocess_function(examples):
    examples["reasoning_trace"] = extract_reasoning_trace(examples["response"])
    return examples

train_dataset = train_dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

In [55]:
for idx, i in enumerate(train_dataset):
    if i['reasoning_trace'] == "":
        print(idx)

In [56]:
for idx, i in enumerate(eval_dataset):
    if i['reasoning_trace'] == "":
        print(idx)

In [6]:
from training_utils import pretrain_tokenize_function
from peft import (
    LoraConfig,
)

model_args = ModelArguments()
training_args = TrainingArguments(output_dir="./output")
data_args = DataArguments()

lora_config = LoraConfig(
    r=model_args.lora_r,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [7]:
model_args.model_name_or_path = "meta-llama/Llama-3.2-1B"
training_args.bf16 = True
training_args.gradient_checkpointing_kwargs = {"use_reentrant": False}  # manually add this argument in the code
training_args.lm_ratio = 0.0

In [9]:
import huggingface_hub
huggingface_hub.login(token="")

In [10]:
# from transformers import AutoModelForCausalLM
# import torch
# model_name = "meta-llama/Llama-3.2-1B"
# icae = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2", resume_download=True)
# icae = icae.to("cuda")
# icae.config.vocab_size, icae.config.hidden_size

In [11]:
from modeling_icae_multi_span import ICAE
model = ICAE(model_args, training_args, lora_config)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Freezing the decoder...
trainable params: 13899776 || all params: 2485798912 || trainable%: 0.5591673539198975
Enabling gradient checkpointing...


In [12]:
model.training_args.model_max_length

28000

In [37]:
lora_config = LoraConfig(
    r=model_args.lora_r,
    lora_alpha=32,
    lora_dropout=model_args.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)

# Initialize model and send it to CUDA device
model = ICAE(model_args, training_args, lora_config)

Freezing the decoder...
trainable params: 13899776 || all params: 2485798912 || trainable%: 0.5591673539198975
Enabling gradient checkpointing...


In [13]:
train_dataset.features

{'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None),
 'response': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [14]:
from datasets import load_dataset

ds = load_dataset("ankner/gsm8k-CoT")
train_dataset = ds["train"]
eval_dataset = ds["test"]
train_dataset = train_dataset.map(lambda example: {**example, "text": f"{example['question']}\n{example['response']}"}).shuffle(seed=42)
eval_dataset = eval_dataset.map(lambda example: {**example, "text": f"{example['question']}\n{example['response']}"})

memory_size = 128
MEM_TOKENS = list(range(model.vocab_size, model.vocab_size + memory_size))

train_dataset = train_dataset.map(pretrain_tokenize_function, batched=True, batch_size=1, fn_kwargs={"model": model, "mem": MEM_TOKENS, "lm_ratio": training_args.lm_ratio})
eval_dataset = eval_dataset.map(pretrain_tokenize_function, batched=True, fn_kwargs={"model": model, "mem": MEM_TOKENS})   # don't add lm in the dev set.

Map:   0%|          | 0/7465 [00:00<?, ? examples/s]

Map:   0%|          | 0/1316 [00:00<?, ? examples/s]

In [15]:
train_dataset

Dataset({
    features: ['question', 'answer', 'response', 'text', 'input_ids', 'prompt_answer_ids', 'labels'],
    num_rows: 7465
})

In [16]:
train_dataset[0]

{'question': "Four adults with 32 teeth went to the dentist for a checkup after realizing they were having severe tooth pain. They were found to have different numbers of damaged teeth, and each person had some teeth removed. The first person had 1/4 of all his teeth removed, and the second person had 3/8 of his teeth removed, the third person had half of his teeth removed, while the last person only had 4 teeth removed. What's the total number of teeth removed at the dental clinic?",
 'answer': '40',
 'response': 'Each adult has 32 teeth initially.\n\nFor the first person, 1/4 of 32 teeth were removed: 32 × (1/4) = 8 teeth removed.\n\nFor the second person, 3/8 of 32 teeth were removed: 32 × (3/8) = 12 teeth removed.\n\nFor the third person, 1/2 of 32 teeth were removed: 32 × (1/2) = 16 teeth removed.\n\nThe fourth person had exactly 4 teeth removed.\n\nAdding all teeth removed: 8 + 12 + 16 + 4 = 40 teeth.\n\nTherefore, the final answer is 40.',
 'text': "Four adults with 32 teeth wen

In [17]:
train_dataset = train_dataset.select([0])
train_dataset[0]

{'question': "Four adults with 32 teeth went to the dentist for a checkup after realizing they were having severe tooth pain. They were found to have different numbers of damaged teeth, and each person had some teeth removed. The first person had 1/4 of all his teeth removed, and the second person had 3/8 of his teeth removed, the third person had half of his teeth removed, while the last person only had 4 teeth removed. What's the total number of teeth removed at the dental clinic?",
 'answer': '40',
 'response': 'Each adult has 32 teeth initially.\n\nFor the first person, 1/4 of 32 teeth were removed: 32 × (1/4) = 8 teeth removed.\n\nFor the second person, 3/8 of 32 teeth were removed: 32 × (3/8) = 12 teeth removed.\n\nFor the third person, 1/2 of 32 teeth were removed: 32 × (1/2) = 16 teeth removed.\n\nThe fourth person had exactly 4 teeth removed.\n\nAdding all teeth removed: 8 + 12 + 16 + 4 = 40 teeth.\n\nTherefore, the final answer is 40.',
 'text': "Four adults with 32 teeth wen

In [18]:
len(train_dataset[0]['input_ids'])

243

In [19]:
a = []
for i in train_dataset:
    a.append((i['input_ids']))

In [20]:
max(a)

[128000,
 28070,
 12884,
 449,
 220,
 843,
 18311,
 4024,
 311,
 279,
 50351,
 369,
 264,
 1817,
 455,
 1306,
 44114,
 814,
 1051,
 3515,
 15748,
 26588,
 6784,
 13,
 2435,
 1051,
 1766,
 311,
 617,
 2204,
 5219,
 315,
 20727,
 18311,
 11,
 323,
 1855,
 1732,
 1047,
 1063,
 18311,
 7108,
 13,
 578,
 1176,
 1732,
 1047,
 220,
 16,
 14,
 19,
 315,
 682,
 813,
 18311,
 7108,
 11,
 323,
 279,
 2132,
 1732,
 1047,
 220,
 18,
 14,
 23,
 315,
 813,
 18311,
 7108,
 11,
 279,
 4948,
 1732,
 1047,
 4376,
 315,
 813,
 18311,
 7108,
 11,
 1418,
 279,
 1566,
 1732,
 1193,
 1047,
 220,
 19,
 18311,
 7108,
 13,
 3639,
 596,
 279,
 2860,
 1396,
 315,
 18311,
 7108,
 520,
 279,
 29106,
 28913,
 5380,
 4959,
 6822,
 706,
 220,
 843,
 18311,
 15453,
 382,
 2520,
 279,
 1176,
 1732,
 11,
 220,
 16,
 14,
 19,
 315,
 220,
 843,
 18311,
 1051,
 7108,
 25,
 220,
 843,
 25800,
 320,
 16,
 14,
 19,
 8,
 284,
 220,
 23,
 18311,
 7108,
 382,
 2520,
 279,
 2132,
 1732,
 11,
 220,
 18,
 14,
 23,
 315,
 220,
 843,
 